In [1]:
import sys
sys.path.append("C:/Users/user/meepc")
import numpy as np
import pandas as pd
from models import Hankel,Corrhankel,Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
df1 = pd.read_csv('~/data/ctown/dataset03.csv')
df2 = pd.read_csv('~/data/ctown/dataset04.csv')

train_normal = pd.concat((df1,df2[df2['ATT_FLAG']==0]),axis=0,ignore_index=True)
train_attack = df2[df2['ATT_FLAG']==1]

In [3]:
sensors = [col for col in train_normal.columns if col not in ['DATETIME','ATT_FLAG']]

In [4]:
scaler = StandardScaler()
X_normal = pd.DataFrame(index=train_normal.index, columns=sensors, data=scaler.fit_transform(train_normal[sensors]))
X_attack = train_attack[sensors].reset_index().drop(columns=['index'])

In [5]:
hankel = Hankel()
corrhankel = Corrhankel()
lag = 60
stride = 0.5

In [6]:
corr_normal,nolag_normal = corrhankel.fit(X_normal.to_numpy(),lag,stride)

In [7]:
corr_attack,nolag_attack = corrhankel.fit(X_attack.to_numpy(),lag,stride)

In [8]:
df_test = pd.read_csv('~/data/ctown/test_dataset.csv')

# Epasad with 1 cluster and no threshold tuning (training attack included in test data)

In [9]:
test_combined = pd.concat((df_test,train_attack),axis=0)
X_test = pd.DataFrame(index=test_combined.index, columns=sensors, data=scaler.fit_transform(test_combined[sensors]))
corr_test,nolag_test = corrhankel.fit(X_test.to_numpy(),lag,stride)
Y_test = test_combined.loc[:,'ATT_FLAG']

In [10]:
labels = hankel.fit(np.array(Y_test),lag,stride)
y_actual = np.any(labels>0,axis=0).astype(int)

In [11]:
sensor_models = []
sensor_predicted = []
accuracy = []
precision = []
recall = []
fscore = []
for i,sens in enumerate(sensors):
    train_normal = X_normal.loc[:,sens].values
    train_attack = X_attack.loc[:,sens].values
    model = Pipeline()
    model.fit(train_normal,train_attack,lag,stride,optimal_k=1,tune=False,corr_normal=corr_normal[:,i].reshape(nolag_normal,len(X_normal.columns)).T,
              only_corr=True)
    test = X_test.loc[:,sens].values
    y_predicted = model.predict(test,corr_test=corr_test[:,i].reshape(nolag_test,len(X_normal.columns)).T)
    sensor_predicted.append(y_predicted)
    accuracy.append(accuracy_score(y_actual,y_predicted))
    precision.append(precision_score(y_actual,y_predicted))
    recall.append(recall_score(y_actual,y_predicted))
    fscore.append(f1_score(y_actual,y_predicted))
    sensor_models.append(model)    

In [12]:
sensor_predicted = np.asarray(sensor_predicted)
y_predicted = np.any(sensor_predicted,axis=0).astype(int)
y_predicted

array([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1,
       0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [13]:
print("Accuracy ",accuracy_score(y_actual,y_predicted))
print("Precision ",precision_score(y_actual,y_predicted))
print("Recall ",recall_score(y_actual,y_predicted))
print("F1-score ",f1_score(y_actual,y_predicted))

Accuracy  0.8
Precision  0.7735849056603774
Recall  0.8913043478260869
F1-score  0.8282828282828283


In [14]:
# Accuracy  0.8
# Precision  0.7735849056603774
# Recall  0.8913043478260869
# F1-score  0.8282828282828283

In [15]:
print("Accuracy -  Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(accuracy).mean(), np.median(np.asarray(accuracy)),np.asarray(accuracy).min(), np.asarray(accuracy).max()))
print("precision - Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(precision).mean(), np.median(np.asarray(precision)),np.asarray(precision).min(), np.asarray(precision).max()))
print("recall -    Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(recall).mean(), np.median(np.asarray(recall)),np.asarray(recall).min(), np.asarray(recall).max()))
print("f1 -        Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(fscore).mean(), np.median(np.asarray(fscore)),np.asarray(fscore).min(), np.asarray(fscore).max()))

Accuracy -  Mean: 0.5400820793433654 Median : 0.5294117647058824 Min : 0.4235294117647059 Max : 0.6588235294117647
precision - Mean: 0.7513263042746806 Median : 0.9 Min : 0.0 Max : 1.0
recall -    Mean: 0.16683518705763395 Median : 0.15217391304347827 Min : 0.0 Max : 0.3695652173913043
f1 -        Mean: 0.2647579188759229 Median : 0.2592592592592593 Min : 0.0 Max : 0.5396825396825397


# Multiple clusters + No threshold tuning (training attack mixed in test data)

In [16]:
cluster_fscore = []
for k in range(2,8):
    sensor_models = []
    sensor_predicted = []
    accuracy = []
    precision = []
    recall = []
    fscore = []
    for i,sens in enumerate(sensors):
        train_normal = X_normal.loc[:,sens].values
        train_attack = X_attack.loc[:,sens].values
        model = Pipeline()
        model.fit(train_normal,train_attack,lag,stride,optimal_k=k,tune=False,kscore_init='inertia',corr_normal=corr_normal[:,i].reshape(nolag_normal,len(X_normal.columns)).T,
                only_corr=True)
        test = X_test.loc[:,sens].values
        y_predicted = model.predict(test,corr_test=corr_test[:,i].reshape(nolag_test,len(X_normal.columns)).T)
        sensor_predicted.append(y_predicted)
        accuracy.append(accuracy_score(y_actual,y_predicted))
        precision.append(precision_score(y_actual,y_predicted))
        recall.append(recall_score(y_actual,y_predicted))
        fscore.append(f1_score(y_actual,y_predicted))
        sensor_models.append(model)    
    sensor_predicted = np.asarray(sensor_predicted)
    y_predicted = np.any(sensor_predicted,axis=0).astype(int)
    print('------Number of Clusters: ',k,'-----------')
    print("Accuracy ",accuracy_score(y_actual,y_predicted))
    print("Precision ",precision_score(y_actual,y_predicted))
    print("Recall ",recall_score(y_actual,y_predicted))
    print("F1-score ",f1_score(y_actual,y_predicted),'\n')
    cluster_fscore.append(f1_score(y_actual,y_predicted))

------Number of Clusters:  2 -----------
Accuracy  0.7529411764705882
Precision  0.6923076923076923
Recall  0.9782608695652174
F1-score  0.8108108108108107 

------Number of Clusters:  3 -----------
Accuracy  0.6352941176470588
Precision  0.5974025974025974
Recall  1.0
F1-score  0.7479674796747967 

------Number of Clusters:  4 -----------
Accuracy  0.6352941176470588
Precision  0.6
Recall  0.9782608695652174
F1-score  0.743801652892562 

------Number of Clusters:  5 -----------
Accuracy  0.611764705882353
Precision  0.5822784810126582
Recall  1.0
F1-score  0.736 

------Number of Clusters:  6 -----------
Accuracy  0.611764705882353
Precision  0.5822784810126582
Recall  1.0
F1-score  0.736 

------Number of Clusters:  7 -----------
Accuracy  0.5647058823529412
Precision  0.5542168674698795
Recall  1.0
F1-score  0.7131782945736435 



In [17]:
# ------Number of Clusters:  2 -----------
# Accuracy  0.7647058823529411
# Precision  0.703125
# Recall  0.9782608695652174
# F1-score  0.8181818181818182 

# ------Number of Clusters:  3 -----------
# Accuracy  0.6588235294117647
# Precision  0.6133333333333333
# Recall  1.0
# F1-score  0.7603305785123966 

# ------Number of Clusters:  4 -----------
# Accuracy  0.6588235294117647
# Precision  0.6133333333333333
# Recall  1.0
# F1-score  0.7603305785123966 

# ------Number of Clusters:  5 -----------
# Accuracy  0.611764705882353
# Precision  0.5822784810126582
# Recall  1.0
# F1-score  0.736 

# ------Number of Clusters:  6 -----------
# Accuracy  0.6235294117647059
# Precision  0.5897435897435898
# Recall  1.0
# F1-score  0.7419354838709677 

# ------Number of Clusters:  7 -----------
# Accuracy  0.5764705882352941
# Precision  0.5609756097560976
# Recall  1.0
# F1-score  0.71875 



In [18]:
cluster_fscore

[0.8108108108108107,
 0.7479674796747967,
 0.743801652892562,
 0.736,
 0.736,
 0.7131782945736435]

In [19]:
# [0.8181818181818182,
#  0.7603305785123966,
#  0.7603305785123966,
#  0.736,
#  0.7419354838709677,
#  0.71875]

# Epasad with 1 cluster + No threshold tuning (No concat of training and test data)

In [20]:
X_test = pd.DataFrame(index=df_test.index, columns=sensors, data=scaler.fit_transform(df_test[sensors]))
corr_test,nolag_test = corrhankel.fit(X_test.to_numpy(),lag,stride)
Y_test = df_test.loc[:,'ATT_FLAG']

In [21]:
labels = hankel.fit(np.array(Y_test),lag,stride)
y_actual = np.any(labels>0,axis=0).astype(int)

In [22]:
sensor_models = []
sensor_predicted = []
accuracy = []
precision = []
recall = []
fscore = []
for i,sens in enumerate(sensors):
    train_normal = X_normal.loc[:,sens].values
    train_attack = X_attack.loc[:,sens].values
    model = Pipeline()
    model.fit(train_normal,train_attack,lag,stride,optimal_k=1,tune=False,kscore_init='inertia',corr_normal=corr_normal[:,i].reshape(nolag_normal,len(X_normal.columns)).T,
              corr_attack=corr_attack[:,i].reshape(nolag_attack,len(X_attack.columns)).T,only_corr=True)
    test = X_test.loc[:,sens].values
    y_predicted = model.predict(test,corr_test=corr_test[:,i].reshape(nolag_test,len(X_normal.columns)).T)
    sensor_predicted.append(y_predicted)
    accuracy.append(accuracy_score(y_actual,y_predicted))
    precision.append(precision_score(y_actual,y_predicted))
    recall.append(recall_score(y_actual,y_predicted))
    fscore.append(f1_score(y_actual,y_predicted))
    sensor_models.append(model)    

In [23]:
sensor_predicted = np.asarray(sensor_predicted)
y_predicted = np.any(sensor_predicted,axis=0).astype(int)
y_predicted

array([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1,
       0, 1])

In [24]:
print("Accuracy ",accuracy_score(y_actual,y_predicted))
print("Precision ",precision_score(y_actual,y_predicted))
print("Recall ",recall_score(y_actual,y_predicted))
print("F1-score ",f1_score(y_actual,y_predicted))

Accuracy  0.75
Precision  0.6764705882352942
Recall  0.7931034482758621
F1-score  0.7301587301587301


In [25]:
# Accuracy  0.75
# Precision  0.6764705882352942
# Recall  0.7931034482758621
# F1-score  0.7301587301587301

In [26]:
print("Accuracy -  Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(accuracy).mean(), np.median(np.asarray(accuracy)),np.asarray(accuracy).min(), np.asarray(accuracy).max()))
print("precision - Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(precision).mean(), np.median(np.asarray(precision)),np.asarray(precision).min(), np.asarray(precision).max()))
print("recall -    Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(recall).mean(), np.median(np.asarray(recall)),np.asarray(recall).min(), np.asarray(recall).max()))
print("f1 -        Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(fscore).mean(), np.median(np.asarray(fscore)),np.asarray(fscore).min(), np.asarray(fscore).max()))

Accuracy -  Mean: 0.6063611491108071 Median : 0.6029411764705882 Min : 0.5294117647058824 Max : 0.7058823529411765
precision - Mean: 0.5909929863418235 Median : 0.75 Min : 0.0 Max : 1.0
recall -    Mean: 0.10264635124298314 Median : 0.06896551724137931 Min : 0.0 Max : 0.3103448275862069
f1 -        Mean: 0.16873336193803887 Median : 0.12903225806451613 Min : 0.0 Max : 0.4736842105263158


# Multiple clusters + No threshold tuning (No concat of training and test data)

In [27]:
cluster_fscore = []
for k in range(2,8):
    sensor_models = []
    sensor_predicted = []
    accuracy = []
    precision = []
    recall = []
    fscore = []
    for i,sens in enumerate(sensors):
        train_normal = X_normal.loc[:,sens].values
        train_attack = X_attack.loc[:,sens].values
        model = Pipeline()
        model.fit(train_normal,train_attack,lag,stride,optimal_k=k,tune=False,corr_normal=corr_normal[:,i].reshape(nolag_normal,len(X_normal.columns)).T,
                corr_attack=corr_attack[:,i].reshape(nolag_attack,len(X_attack.columns)).T,only_corr=True)
        test = X_test.loc[:,sens].values
        y_predicted = model.predict(test,corr_test=corr_test[:,i].reshape(nolag_test,len(X_normal.columns)).T)
        sensor_predicted.append(y_predicted)
        accuracy.append(accuracy_score(y_actual,y_predicted))
        precision.append(precision_score(y_actual,y_predicted))
        recall.append(recall_score(y_actual,y_predicted))
        fscore.append(f1_score(y_actual,y_predicted))
        sensor_models.append(model)    
    sensor_predicted = np.asarray(sensor_predicted)
    y_predicted = np.any(sensor_predicted,axis=0).astype(int)
    print('------Number of Clusters: ',k,'-----------')
    print("Accuracy ",accuracy_score(y_actual,y_predicted))
    print("Precision ",precision_score(y_actual,y_predicted))
    print("Recall ",recall_score(y_actual,y_predicted))
    print("F1-score ",f1_score(y_actual,y_predicted),'\n')
    cluster_fscore.append(f1_score(y_actual,y_predicted))

------Number of Clusters:  2 -----------
Accuracy  0.75
Precision  0.6363636363636364
Recall  0.9655172413793104
F1-score  0.7671232876712328 

------Number of Clusters:  3 -----------
Accuracy  0.6176470588235294
Precision  0.5272727272727272
Recall  1.0
F1-score  0.6904761904761904 

------Number of Clusters:  4 -----------
Accuracy  0.5441176470588235
Precision  0.48333333333333334
Recall  1.0
F1-score  0.6516853932584269 

------Number of Clusters:  5 -----------
Accuracy  0.5441176470588235
Precision  0.48333333333333334
Recall  1.0
F1-score  0.6516853932584269 

------Number of Clusters:  6 -----------
Accuracy  0.5735294117647058
Precision  0.5
Recall  1.0
F1-score  0.6666666666666666 

------Number of Clusters:  7 -----------
Accuracy  0.5294117647058824
Precision  0.47540983606557374
Recall  1.0
F1-score  0.6444444444444445 



In [28]:
# ------Number of Clusters:  2 -----------
# Accuracy  0.75
# Precision  0.6363636363636364
# Recall  0.9655172413793104
# F1-score  0.7671232876712328 

# ------Number of Clusters:  3 -----------
# Accuracy  0.6470588235294118
# Precision  0.5471698113207547
# Recall  1.0
# F1-score  0.7073170731707317 

# ------Number of Clusters:  4 -----------
# Accuracy  0.5441176470588235
# Precision  0.4827586206896552
# Recall  0.9655172413793104
# F1-score  0.6436781609195403 

# ------Number of Clusters:  5 -----------
# Accuracy  0.5588235294117647
# Precision  0.4915254237288136
# Recall  1.0
# F1-score  0.6590909090909091 

# ------Number of Clusters:  6 -----------
# Accuracy  0.5294117647058824
# Precision  0.47540983606557374
# Recall  1.0
# F1-score  0.6444444444444445 

# ------Number of Clusters:  7 -----------
# Accuracy  0.5294117647058824
# Precision  0.47540983606557374
# Recall  1.0
# F1-score  0.6444444444444445 



In [29]:
cluster_fscore

[0.7671232876712328,
 0.6904761904761904,
 0.6516853932584269,
 0.6516853932584269,
 0.6666666666666666,
 0.6444444444444445]

In [30]:
# [0.7671232876712328,
#  0.7073170731707317,
#  0.6436781609195403,
#  0.6590909090909091,
#  0.6444444444444445,
#  0.6444444444444445]

# Multiple clusters + Threshold tuning (No concat of training and test data)

In [31]:
cluster_fscore = []
for k in range(2,8):
    sensor_models = []
    sensor_predicted = []
    accuracy = []
    precision = []
    recall = []
    fscore = []
    for i,sens in enumerate(sensors):
        train_normal = X_normal.loc[:,sens].values
        train_attack = X_attack.loc[:,sens].values
        model = Pipeline()
        model.fit(train_normal,train_attack,lag,stride,optimal_k=k,kscore_init='inertia',corr_normal=corr_normal[:,i].reshape(nolag_normal,len(X_normal.columns)).T,
                corr_attack=corr_attack[:,i].reshape(nolag_attack,len(X_attack.columns)).T,only_corr=True)
        test = X_test.loc[:,sens].values    
        y_predicted = model.predict(test,corr_test=corr_test[:,i].reshape(nolag_test,len(X_normal.columns)).T)
        sensor_predicted.append(y_predicted)
        accuracy.append(accuracy_score(y_actual,y_predicted))
        precision.append(precision_score(y_actual,y_predicted))
        recall.append(recall_score(y_actual,y_predicted))
        fscore.append(f1_score(y_actual,y_predicted))
        sensor_models.append(model)    
    sensor_predicted = np.asarray(sensor_predicted)
    y_predicted = np.any(sensor_predicted,axis=0).astype(int)
    print('------Number of Clusters: ',k,'-----------')
    print("Accuracy ",accuracy_score(y_actual,y_predicted))
    print("Precision ",precision_score(y_actual,y_predicted))
    print("Recall ",recall_score(y_actual,y_predicted))
    print("F1-score ",f1_score(y_actual,y_predicted),'\n')
    cluster_fscore.append(f1_score(y_actual,y_predicted))

------Number of Clusters:  2 -----------
Accuracy  0.75
Precision  0.6363636363636364
Recall  0.9655172413793104
F1-score  0.7671232876712328 

------Number of Clusters:  3 -----------
Accuracy  0.6323529411764706
Precision  0.5370370370370371
Recall  1.0
F1-score  0.6987951807228916 

------Number of Clusters:  4 -----------
Accuracy  0.6323529411764706
Precision  0.5370370370370371
Recall  1.0
F1-score  0.6987951807228916 

------Number of Clusters:  5 -----------
Accuracy  0.5882352941176471
Precision  0.5087719298245614
Recall  1.0
F1-score  0.6744186046511628 

------Number of Clusters:  6 -----------
Accuracy  0.5441176470588235
Precision  0.48333333333333334
Recall  1.0
F1-score  0.6516853932584269 

------Number of Clusters:  7 -----------
Accuracy  0.5
Precision  0.4603174603174603
Recall  1.0
F1-score  0.6304347826086957 



In [32]:
# ------Number of Clusters:  2 -----------
# Accuracy  0.75
# Precision  0.6363636363636364
# Recall  0.9655172413793104
# F1-score  0.7671232876712328 

# ------Number of Clusters:  3 -----------
# Accuracy  0.6323529411764706
# Precision  0.5370370370370371
# Recall  1.0
# F1-score  0.6987951807228916 

# ------Number of Clusters:  4 -----------
# Accuracy  0.6176470588235294
# Precision  0.5272727272727272
# Recall  1.0
# F1-score  0.6904761904761904 

# ------Number of Clusters:  5 -----------
# Accuracy  0.45588235294117646
# Precision  0.4393939393939394
# Recall  1.0
# F1-score  0.6105263157894737 

# ------Number of Clusters:  6 -----------
# Accuracy  0.5735294117647058
# Precision  0.5
# Recall  1.0
# F1-score  0.6666666666666666 

# ------Number of Clusters:  7 -----------
# Accuracy  0.4852941176470588
# Precision  0.453125
# Recall  1.0
# F1-score  0.6236559139784946 



In [33]:
cluster_fscore

[0.7671232876712328,
 0.6987951807228916,
 0.6987951807228916,
 0.6744186046511628,
 0.6516853932584269,
 0.6304347826086957]

In [ ]:
# [0.7671232876712328,
#  0.6987951807228916,
#  0.6904761904761904,
#  0.6105263157894737,
#  0.6666666666666666,
#  0.6236559139784946]